# (OpenCV - Chap9) 변환영역 처리
> 공간 주파수의 이해, 이산/고속 푸리에 변환, FFT, 이산 코사인변환
- toc: true
- branch: master
- badges: false
- comments: true
- author: dinonene
- categories: [python]

## 9.1 공간 주파수의 이해

공간주파수: 밝기의 변화 정도에 따라서 고주파 영역 / 저주파 영역으로 분류

`-` 저주파 공간 영영
- 화소 밝기가 거의 변화가 없거나 점진적으로 변화하는 것
- 영상에서 배경 부분이나 객체의 내부에 많이 있음


`-` 고주파 공간 영역
- 화소 밝기가 급변하는 것
- 경계부분이나 객체의 모서리 부분

`-` 영상을 주파수 영역별로 분리 한다면?
- 경계부분에 많은 고주파 성분 제거한 영상 $\to$ 경계 흐려진 영상
- 고주파 성분만 취한 영상 $\to$ 경계나 모서리만 포함하는 영상. 즉, 에지 영상

## 9.2 이산 푸리에 변환

### 신호의 기본 전제
주기를 가진 신호는 ***정현파(사인파) / 여현파(코사인파)***의 합으로 표현할 수 있다.

`-` 정현파 / 여현파
- 모든 파형 중에 가장 순수한 파형을 말하는 것으로 사인 또는 코사인 함수로 된  신호

`-` 예) 주기를 갖는 신호
$$g(t) = 0.3\cdot g_1(t) - 0.7\cdot g_2(t) + 0.6\cdot g_3(t)$$

`-` 여러 사인 및 코사인 함수들로 분리 가능
- 분리된 신호(기저 함수): $g_1(t)=sin(2\pi\cdot t),\space g_2(t) = sin(2\pi\cdot 3t),\space g_3(t) = sin(2\pi\cdot 5t)$


- 기저 함수에 곱해지는 값(주파수 계수) : $0.3, -0.7, 0.5$) $\leftarrow$주파수 성분의 크기


### 주파수 변환 수식
$$g(t) = \int_{-\infty}^{\infty}G(f)\cdot g_f(t)df$$

- $g_f(t)$ : f주파수에 대한 기저함수
- $G(t)$ 기저함수의 계수

모든 주파수에 대한 기저함수와 그 계수들의 선형 조합

####  로그적용 정규화 함수

In [1]:
def calc_spectrum(complex):
    if complex.ndim==2:
        dst = abs(complex)
    else:
        dst = cv2.magnitude(complex[:,:,0], complex[:,:,1])
    dst = cv2.log(dst + 1)
    cv2.normalize(dst, dst, 0, 255, cv2.NORM_MINMAX)
    return cv2.convertScaleAbs(dst)

#### 주파수 스펙트럼 영상
- 저주파 영영이 모서리 부분에 위치, 고주파 부분이 중심부에 위치
- 사각형의 각 모서리를 중심으로 원형의 밴드를 형성하여 주파수 영역 분포
    - 해당 주파수 영역 처리시 불편함 $\to$ 모양 변경 필요

#### 해결방법 - 셔플링(shuffling) / 시프트(shift) 연산
좌측 상단이 1사분면, 우측상단이 2사분면, 우측 하단 3사분면, 좌픅 하당 4사분면

- 1사분면과 3사분면 맞교환, 2사분면과 4사분면 맞교환

In [2]:
# 셔플링 함수 구현
def fftshift(img):
    dst = np.zeros(img.shape, img.dtype)
    h, w = dst.shape[:2]
    cy, cx = h//2, w//2
    dst[h-cy:, w-cx:] = np.copy(img[0:cy, 0:cx]) #1사분면 -> 3사분면

## 9.3 고속 푸리에 변환

#### 푸리에 변환 수식 - 짝수부/홀수부 분리 가능
$$G(k) = \sum_{n=0}^{L-1}g[2n]\cdot e^{-j2\pi k \frac{2n}{2L}} + \sum_{n=0}^{L-1}g[2n+1]\cdot e^{-j2\pi k \frac{2n+1}{2L}}$$